# Inference Pipeline

In [2]:
import pandas as pd
from modules.ontology_classes import CogafInstance
from modules.emotion import get_emotion
from modules.engine import COGN_FUNC
from modules.text_processing import load_text_files
from owlready2 import *

## Load and process texts

In [3]:
text_list = list(load_text_files('./../data/text/raw/').values())

In [5]:
# Load preprocess model
import joblib
MODELS_PATH = "./models/"
processor = joblib.load(MODELS_PATH + "preprocessor.pk")


In [6]:
# First step of processor if pipeline is text cleaning. Clean text is needed for emotion recognition
clean_texts = processor[0].transform(text_list)

# Subsequent steps transforms the clean text into numerical representation
X = pd.DataFrame(processor[1:].transform(clean_texts))

In [7]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.018390,0.153832,-0.036187,-0.096114,0.000189,0.008908,0.025292,-0.015457,-0.002550,0.051419,...,0.004520,-0.021593,0.034531,0.036302,0.009767,-0.002026,-0.014995,-0.014123,-0.044726,-0.007895
1,-0.026766,0.421393,-0.068882,0.105197,0.042154,0.029687,0.110361,-0.105812,-0.130303,0.160647,...,0.005570,-0.002325,-0.000482,-0.014414,-0.022651,-0.008666,-0.001102,0.011631,-0.002150,0.002202
2,0.011969,0.184345,-0.021444,-0.121609,-0.055329,-0.001112,0.375440,-0.127316,-0.151340,-0.158765,...,0.004352,0.011736,0.001587,0.005040,0.001057,0.003236,0.006027,-0.008859,-0.001709,0.000355
3,-0.081711,0.261403,-0.040354,-0.218305,0.017647,-0.012647,0.264102,0.090078,-0.115739,-0.180781,...,-0.002372,-0.005515,0.006456,0.006268,-0.002655,-0.004133,-0.000517,0.003722,0.007989,0.005342
4,-0.051404,0.201853,-0.044697,-0.084711,0.000786,0.010653,0.172421,-0.080946,-0.096627,0.077496,...,-0.011919,-0.002356,-0.002698,-0.006592,-0.002185,-0.009310,0.006778,-0.018198,-0.016595,0.014457


## Cognitive Function Inference

In [8]:
# Load model and predict cognitive function
clf = joblib.load(MODELS_PATH + "svm.pk")
y_pred = clf.predict(X)
y_pred

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6], dtype=int64)

## Emotion Recognition

In [ ]:
# TODO: Save hourglass model dimension values and map through G(x) (inverted bell curve)

In [13]:
results = []
for i in range(len(clean_texts)):
    response = get_emotion(clean_texts[i])
    result = {
        "cognitive_function": COGN_FUNC[y_pred[i]],
        "status_code": response.status_code,
        "content": response.content.decode()
    }
    if response.status_code == 200:
        
        result.update({"emotion": re.search(r'[a-z]*', result["content"]).group(0)})
    results.append(result)

In [33]:
results[5]["content"]

'ecstasy (89.3%) & serenity (-27.11%) [INTROSPECTION=95.47%,TEMPER=-8.95%,ATTITUDE=-18.4%,SENSITIVITY=-99.0%]\n'

In [34]:
clean_texts[5]

'leccion aprendido caida altura montaje estructura caido altura paso estructura inicial faltar colocar peldano tramo numero analizar estructura accion colaborador anclado punto x retirar tornillo soporte peldano encajar reemplazando el punzon acero resistir peso viga soltar el tornillo maniobra paso 2 peso viga recaar soporte superior reventar soldadura generar caer viga halir colaborador tramo escalera recorrer 2.5 metro altura causar falencia proceso seleccion capacitacion definicion rol responsabilidad trabajador deficiencia analisis tarea tener identificado riesgo tarea izaje carga peso viga resistencia soldadura soltar el queda inestable peso recaar punto soldadura superior reventandolo identificado permiso altura analisis seguro tarea punto anclaje correcto utilizacion elemento compatible estructura montado evitar control ejecucion trabajo profundidad planeacion trabajo fortalecer conocimiento tecnico fortalecer aplicacion herramienta analisis riesgo claridad rol responsabilidad 

In [35]:
get_emotion("""leccion aprendido tipo accidente caida altura fatalidad actividad subir silo escalera vertical cargo despachador producto granel descripcion accidente trabajador recientemente incorporado empresa 9 subir escalera vertical silo disponer resguardo seguridad dispositivo anti caido ascenso perder equilibrio caer altura aproximado 8 metro producir el fallecimiento posterior trabajador accion preventivo proveer induccion personal riesgo expuesto lugar trabajo trabajo altura deber debidamente comunicar autorizado procedimiento trabajo seguro ej. permiso trabajo escalera servicio deber disponer proteccion colectivo resguardo seguridad tipo aro metalico protector linea vida trabajador usar ames cuerpo completo evaluar necesidad instalar mecanismo bloqueo impedir fisicamente acceso personal autorizado area riesgo normativa legal art 11 d.e.2393 obligacion empleadores.9 instruir riesgo puesto trabajo forma metodo prevenir el personal ingresar laborar empresa art 26 d.e.2393 escalera fijo servicio.3 emplear escalera fijo altura mayor 7 metro instalar plataforma descanso 7 metro fraccion estar provisto aro metalico protector separacion maximo 500 milimetro dispositivo anti caida distancia maximo caida libre metro art 187.- d.e.2393 prohibicion empleador h permitir trabajador realizar labor riesgo entrenar previamente""").content

b'ecstasy (34.44%) & delight (26.78%) [INTROSPECTION=77.37%,TEMPER=36.53%,ATTITUDE=74.84%,SENSITIVITY=-59.0%]\n'

In [25]:
import re

result = re.search(r'[a-z]*', results[0]["content"].decode())
print(result.group(0))

pleasantness


## Ontology components inference

In [ ]:
#TODO: Infere related components through ontology

for result in results:
    if result.status_code == 200:
        result.update({"cogaf_instance": CogafInstance(result["cognitive_function"], result["emotion"])})